In [1]:
import sys
sys.path.append('/home/rrg0013@auburn.edu/git/nodules/src/')

In [2]:
import lidc
import os 
import SimpleITK as sitk
import pandas as pd
import numpy as np
from collections import defaultdict
from glob import glob

In [3]:
BINNO = 1
LUNA_PATH = '/scr/data/nodules/luna/train/subset'+str(BINNO)+'/'
CAND_DAT_PATH = '/scr/data/nodules/testseg/sega/one.txt'
CAND_PATH = '/scr/data/nodules/testseg/sega/backups2/one/'
XMLDIR = '/home/rrg0013@auburn.edu/xmlb/xml/'
OUT_PATH = '/scr/data/nodules/testseg/cands/relabeled/one/'

In [4]:
df = pd.read_csv(CAND_DAT_PATH,header=None)
#df.head

In [5]:
#df = df[df.iloc[:,1]=='t']
#cdf = df.iloc[:,[3,4,5,6,7,8]]
#print cdf.head
cdf = df.iloc[:,[2,6,7,8,9,10,11,12,13,14]]
dat = cdf.as_matrix()

In [6]:
uids = df.iloc[:,2]
uids = set(uids)
uids = list(uids)
cands = {}
for uid in uids:
  tdf = df[df.iloc[:,2] == uid]
  tdf = tdf.iloc[:,6:15]
  cands[uid] = tdf
fnames = defaultdict(list)
files = glob(CAND_PATH+'/*.png')
for fpath in files:
  fname = fpath.split('-')
  fname = fname[-1]
  fname = fname[:-4]
  fnames[fname].append(fpath)
  #print fpath

In [7]:
def run():
  ct = 0
  for parent, subdir, files in os.walk(LUNA_PATH):
    for fname in files:
      if ".mhd" in fname.lower():
        fpath = os.path.join(parent,fname)
        itk = sitk.ReadImage(fpath)
        origin = np.array(itk.GetOrigin())
        spacing = np.array(itk.GetSpacing())
        fname = fname[:-4]
        scan = lidc.xml(XMLDIR,fname,spacing,origin)
        lidc.cluster(scan)
        scan.combine()
        cand = cands[fname]
        fpaths = fnames[fname]
        tcoords = cand.iloc[:,0:3].as_matrix()
        coords = []
        for i in range(tcoords.shape[0]):
          coords.append((tcoords[i,2],tcoords[i,1],tcoords[i,0]))
        for nodule in scan.nodules:
          centroid = nodule.centroid
          d = nodule.diameter
          for coord in coords:
            dist = scan.getDistance(centroid,coord)
            if d*.75 > dist:
              print "found"
              tdf = cand[cand.iloc[:,0]==coord[2]] 
              tdf = tdf[tdf.iloc[:,1]==coord[1]]
              tdf = tdf[tdf.iloc[:,2]==coord[0]]
              print tdf
              tm = tdf.as_matrix()
              cx,cy,cz,ca,cb,cc,cd,ce,cf = tm[0,0],tm[0,1],tm[0,2],tm[0,3],tm[0,4],tm[0,5],tm[0,6],tm[0,7],tm[0,8]
              for fpath in fpaths:
                fvals = fpath.split('_')
                fvals = fvals[-1]
                fvals = fvals.split('-')
                try:
                  a,b,c,d,e,f,_,__ = fvals
                except ValueError:
                  try:
                    a,b,c,d,_,e,__,f,___,____ = fvals
                  except ValueError:
                    a,b,c,d,e,_,f,__,___ = fvals
                if int(float(ca)) == int(float(a)):
                  if int(float(cb)) == int(float(b)):
                    if int(float(cc)) == int(float(c)):
                      if float(cd) == float(d):
                        if abs(int(float(ce))) == abs(int(float(e))):
                          if abs(int(float(cf))) == abs(int(float(f))):
                            print "OLD FILEPATH"
                            print fpath
                            ct += 1
                            print ct
                            npath = OUT_PATH + '/t,b_' + str(ct) + '_' + str(BINNO) + '.png'
                            os.rename(fpath,npath)
                          else:  
                            print "\n\n\nERROR F\n\n\n"
                            print abs(float(cf))
                            print abs(float(f))
                        else: 
                          print "\n\n\nERROR E\n\n\n"
                          print ce
                          print e
                #rename to 'newdir/t,b,...'
              coords.remove(coord)
              break
  print "TOTAL" + str(ct)
  return ct

In [8]:
ct = run()

/home/rrg0013@auburn.edu/xmlb/xml/
/home/rrg0013@auburn.edu/xmlb/xml//1.3.6.1.4.1.14519.5.2.1.6279.6001.100684836163890911914061745866.xml
1.3.6.1.4.1.14519.5.2.1.6279.6001.100684836163890911914061745866
/home/rrg0013@auburn.edu/xmlb/xml/
/home/rrg0013@auburn.edu/xmlb/xml//1.3.6.1.4.1.14519.5.2.1.6279.6001.104562737760173137525888934217.xml
1.3.6.1.4.1.14519.5.2.1.6279.6001.104562737760173137525888934217
[  97.61909404  214.57096678  103.6896972 ]
found
     6    7   8    9     10   11   12    13          14
14  104  215  97  669  1200  1.0  1.0  2381  823.662182
OLD FILEPATH
/scr/data/nodules/testseg/sega/backups2/one/t_669-1200-1.0-0.999999999287-2381-823.662182362-3257-1.3.6.1.4.1.14519.5.2.1.6279.6001.104562737760173137525888934217.png
1
/home/rrg0013@auburn.edu/xmlb/xml/
/home/rrg0013@auburn.edu/xmlb/xml//1.3.6.1.4.1.14519.5.2.1.6279.6001.106719103982792863757268101375.xml
1.3.6.1.4.1.14519.5.2.1.6279.6001.106719103982792863757268101375
[ 105.06734057  215.78610313  186.17476678]


/home/rrg0013@auburn.edu/xmlb/xml/
/home/rrg0013@auburn.edu/xmlb/xml//1.3.6.1.4.1.14519.5.2.1.6279.6001.139595277234735528205899724196.xml
1.3.6.1.4.1.14519.5.2.1.6279.6001.139595277234735528205899724196
[ 113.33858268  142.41732283  228.79527559]
found
      6    7    8   9   10   11        12   13          14
924  231  144  113  46  80  1.0  0.999965 -499 -642.630435
OLD FILEPATH
/scr/data/nodules/testseg/sega/backups2/one/f_46-80-1.0-0.999965312688--499--642.630434783-277-1.3.6.1.4.1.14519.5.2.1.6279.6001.139595277234735528205899724196.png
18
/home/rrg0013@auburn.edu/xmlb/xml/
/home/rrg0013@auburn.edu/xmlb/xml//1.3.6.1.4.1.14519.5.2.1.6279.6001.140527383975300992150799777603.xml
1.3.6.1.4.1.14519.5.2.1.6279.6001.140527383975300992150799777603
[ 224.0625      128.70833333  190.45833333]
found
      6    7    8   9   10   11        12   13          14
949  191  129  224  45  64  1.0  0.999776  148 -324.911111
OLD FILEPATH
/scr/data/nodules/testseg/sega/backups2/one/f_45-64-1.0-0.99977

/home/rrg0013@auburn.edu/xmlb/xml/
/home/rrg0013@auburn.edu/xmlb/xml//1.3.6.1.4.1.14519.5.2.1.6279.6001.169128136262002764211589185953.xml
1.3.6.1.4.1.14519.5.2.1.6279.6001.169128136262002764211589185953
[ 220.18220609  215.5798706   301.5020316 ]
[ 295.16822809  224.22777089  207.51664152]
[  92.77544396  241.0698224    95.68795805]
[ 245.46227804  170.12282288  258.23708086]
[ 117.26592848  256.77810349  254.41818796]
[ 144.          244.14285714  264.        ]
[ 141.57789661  233.31764706  264.06060606]
[ 119.04545455  250.63636364  271.        ]
[ 330.89906503  195.31367112  124.14131447]
[ 329.29551821  184.6022409   130.57422969]
found
       6    7    8    9     10   11        12   13         14
1967  305  215  223  673  1800  1.0  0.999033  303 -243.02526
found
       6    7    8   9    10   11   12   13          14
1968  208  225  295  96  180  1.0  1.0 -360 -548.322917
OLD FILEPATH
/scr/data/nodules/testseg/sega/backups2/one/t_96-180-1.0-1.0--360--548.322916667-403-1.3.6.1.4.

/home/rrg0013@auburn.edu/xmlb/xml/
/home/rrg0013@auburn.edu/xmlb/xml//1.3.6.1.4.1.14519.5.2.1.6279.6001.183843376225716802567192412456.xml
1.3.6.1.4.1.14519.5.2.1.6279.6001.183843376225716802567192412456
[ 306.85319173  241.64039057  192.45652696]
[ 249.70934615  313.32989119  206.69535629]
[ 282.44523916  225.18271475  227.91090739]
[ 276.02789171  286.94852338  278.50984413]
[ 254.56484205  203.45025569  270.93048747]
[  95.89473684  174.44661654  206.54285714]
[ 135.52380952  251.04761905  294.14285714]
found
       6    7    8    9    10   11        12  13          14
2358  193  242  307  427  648  1.0  0.998598  68 -285.187354
OLD FILEPATH
/scr/data/nodules/testseg/sega/backups2/one/t_427-648-1.0-0.998597803803-68--285.18735363-914-1.3.6.1.4.1.14519.5.2.1.6279.6001.183843376225716802567192412456.png
57
found
       6    7    8    9    10   11        12  13          14
2359  207  314  251  186  336  1.0  0.999911  43 -256.403226
OLD FILEPATH
/scr/data/nodules/testseg/sega/backups2/

/home/rrg0013@auburn.edu/xmlb/xml/
/home/rrg0013@auburn.edu/xmlb/xml//1.3.6.1.4.1.14519.5.2.1.6279.6001.231002159523969307155990628066.xml
1.3.6.1.4.1.14519.5.2.1.6279.6001.231002159523969307155990628066
[ 268.11505042  193.21599441  222.22240994]
[ 253.3203789   196.78778802  212.56216078]
[ 105.77774573  261.42987215  164.59758012]
[ 240.14583333  182.          228.41666667]
found
       6    7    8    9    10   11   12    13          14
3102  222  193  268  298  512  1.0  1.0  2488  584.872483
OLD FILEPATH
/scr/data/nodules/testseg/sega/backups2/one/t_298-512-1.0-0.9999999944-2488-584.872483221-3503-1.3.6.1.4.1.14519.5.2.1.6279.6001.231002159523969307155990628066.png
72
found
       6    7    8   9   10   11   12    13          14
3103  214  198  254  56  64  1.0  1.0  2067  100.714286
OLD FILEPATH
/scr/data/nodules/testseg/sega/backups2/one/t_56-64-1.0-1.0-2067-100.714285714-2949-1.3.6.1.4.1.14519.5.2.1.6279.6001.231002159523969307155990628066.png
73
found
       6    7    8   9   

/home/rrg0013@auburn.edu/xmlb/xml/
/home/rrg0013@auburn.edu/xmlb/xml//1.3.6.1.4.1.14519.5.2.1.6279.6001.259018373683540453277752706262.xml
1.3.6.1.4.1.14519.5.2.1.6279.6001.259018373683540453277752706262
[ 149.21428571  296.82142857  128.60714286]
found
       6    7    8   9   10   11   12  13          14
3429  130  297  150  52  80  1.0  1.0   6 -355.903846
OLD FILEPATH
/scr/data/nodules/testseg/sega/backups2/one/f_52-80-1.0-1.0-6--355.903846154-882-1.3.6.1.4.1.14519.5.2.1.6279.6001.259018373683540453277752706262.png
93
/home/rrg0013@auburn.edu/xmlb/xml/
/home/rrg0013@auburn.edu/xmlb/xml//1.3.6.1.4.1.14519.5.2.1.6279.6001.259543921154154401875872845498.xml
1.3.6.1.4.1.14519.5.2.1.6279.6001.259543921154154401875872845498
[ 110.16777839  137.64177087  159.82528803]
[ 103.53692756  117.81526645  150.40614608]
[ 109.93144857  100.6848866   143.97975952]
found
       6    7    8    9    10   11        12  13       14
3481  160  138  111  125  216  1.0  0.999888  35 -426.832
found
       6

/home/rrg0013@auburn.edu/xmlb/xml/
/home/rrg0013@auburn.edu/xmlb/xml//1.3.6.1.4.1.14519.5.2.1.6279.6001.300271604576987336866436407488.xml
ATTRIBUTE ERROR
1.3.6.1.4.1.14519.5.2.1.6279.6001.300271604576987336866436407488
[ 256.   90.   75.]
found
      6   7    8   9    10   11   12  13          14
4053  75  90  256  47  125  1.0  1.0  44 -431.723404
OLD FILEPATH
/scr/data/nodules/testseg/sega/backups2/one/f_47-125-1.0-1.0-44--431.723404255-945-1.3.6.1.4.1.14519.5.2.1.6279.6001.300271604576987336866436407488.png
112
/home/rrg0013@auburn.edu/xmlb/xml/
/home/rrg0013@auburn.edu/xmlb/xml//1.3.6.1.4.1.14519.5.2.1.6279.6001.306948744223170422945185006551.xml
1.3.6.1.4.1.14519.5.2.1.6279.6001.306948744223170422945185006551
[ 280.94721136  184.86867302  226.82075364]
[ 296.91387835  168.39852811  124.35657152]
found
       6    7    8     9     10   11       12   13         14
4092  226  186  281  1043  2448  1.0  0.99918  122 -53.869607
OLD FILEPATH
/scr/data/nodules/testseg/sega/backups2/one/

/home/rrg0013@auburn.edu/xmlb/xml/
/home/rrg0013@auburn.edu/xmlb/xml//1.3.6.1.4.1.14519.5.2.1.6279.6001.334105754605642100456249422350.xml
1.3.6.1.4.1.14519.5.2.1.6279.6001.334105754605642100456249422350
[  46.8400641   172.86185897  127.08306624]
found
       6    7   8   9   10   11   12    13          14
4521  128  173  47  21  36  1.0  1.0  1813  861.238095
OLD FILEPATH
/scr/data/nodules/testseg/sega/backups2/one/t_21-36-1.0-1.0-1813-861.238095238-2130-1.3.6.1.4.1.14519.5.2.1.6279.6001.334105754605642100456249422350.png
128
/home/rrg0013@auburn.edu/xmlb/xml/
/home/rrg0013@auburn.edu/xmlb/xml//1.3.6.1.4.1.14519.5.2.1.6279.6001.335866409407244673864352309754.xml
1.3.6.1.4.1.14519.5.2.1.6279.6001.335866409407244673864352309754
[  61.84444444  163.5537037   173.91234568]
found
       6    7   8   9    10   11   12   13          14
4630  173  164  62  69  125  1.0  1.0  163 -376.072464
OLD FILEPATH
/scr/data/nodules/testseg/sega/backups2/one/f_69-125-1.0-1.0-163--376.072463768-1055-1.3.

/home/rrg0013@auburn.edu/xmlb/xml/
/home/rrg0013@auburn.edu/xmlb/xml//1.3.6.1.4.1.14519.5.2.1.6279.6001.655242448149322898770987310561.xml
1.3.6.1.4.1.14519.5.2.1.6279.6001.655242448149322898770987310561
[ 133.41761157  187.34002632  105.55196578]
[ 138.52873563  185.8045977   114.71264368]
found
       6    7    8   9    10   11       12  13          14
5116  105  185  128  89  180  1.0  0.99251  69 -194.898876
OLD FILEPATH
/scr/data/nodules/testseg/sega/backups2/one/f_89-180-1.0-0.992510329137-69--194.898876404-955-1.3.6.1.4.1.14519.5.2.1.6279.6001.655242448149322898770987310561.png
144
found
       6    7    8    9    10   11   12   13          14
5117  116  186  138  264  392  1.0  1.0  335 -192.916667
OLD FILEPATH
/scr/data/nodules/testseg/sega/backups2/one/f_264-392-1.0-1.0-335--192.916666667-1125-1.3.6.1.4.1.14519.5.2.1.6279.6001.655242448149322898770987310561.png
145
/home/rrg0013@auburn.edu/xmlb/xml/
/home/rrg0013@auburn.edu/xmlb/xml//1.3.6.1.4.1.14519.5.2.1.6279.6001.66301925

1.3.6.1.4.1.14519.5.2.1.6279.6001.970264865033574190975654369557
[  57.66180544  187.5581555   124.64212211]
[ 256.33216685  135.50433931   63.30949048]
[  30.86486486  148.27027027  111.56756757]
[ 198.50909091   97.58181818  210.12727273]
found
       6    7   8   9    10   11   12   13      14
5732  125  188  58  50  100  1.0  1.0  897 -318.28
OLD FILEPATH
/scr/data/nodules/testseg/sega/backups2/one/t_50-100-1.0-1.0-897--318.28-1727-1.3.6.1.4.1.14519.5.2.1.6279.6001.970264865033574190975654369557.png
157
found
      6    7    8   9   10   11   12   13     14
5733  64  136  256   6   8  1.0  1.0  313  124.5
found
       6    7   8   9   10   11   12  13     14
5752  112  148  31  10  12  1.0  1.0  75 -255.0
TOTAL157


In [178]:
print ct

157
